### 03 - Treinamento dos Modelos

Treina os modelos do artigo + melhorias propostas

# Imports

In [21]:
import subprocess
import sys
sys.path.append('/home/jovyan/work')

import importlib
import src.mlflow_utils
importlib.reload(src.mlflow_utils)
from src import MLFlowClient

print("✅ MLFlowClient atualizado!")

! pip install xgboost

import pandas as pd
import numpy as np
from src import S3Client, ModelTrainer, MLFlowClient

✅ MLFlowClient atualizado!


# Carregar dados processados

In [22]:
s3 = S3Client()

X_train = s3.read_csv('processed/X_train_scaled.csv')
X_test = s3.read_csv('processed/X_test_scaled.csv')
y_train = s3.read_csv('processed/y_train.csv')['target']
y_test = s3.read_csv('processed/y_test.csv')['target']

print(f"📊 X_train: {X_train.shape}")
print(f"📊 X_test: {X_test.shape}")
print(f"📊 y_train: {y_train.shape}")
print(f"📊 y_test: {y_test.shape}")

INFO:src.s3_utils:S3 Client inicializado: http://minio:9000/ml-bucket-heart
INFO:src.s3_utils:✅ CSV lido: s3://ml-bucket-heart/processed/X_train_scaled.csv (602 linhas)
INFO:src.s3_utils:✅ CSV lido: s3://ml-bucket-heart/processed/X_test_scaled.csv (232 linhas)
INFO:src.s3_utils:✅ CSV lido: s3://ml-bucket-heart/processed/y_train.csv (602 linhas)
INFO:src.s3_utils:✅ CSV lido: s3://ml-bucket-heart/processed/y_test.csv (232 linhas)


📊 X_train: (602, 19)
📊 X_test: (232, 19)
📊 y_train: (602,)
📊 y_test: (232,)


# Inicializar trainer e MLFlow

In [23]:
trainer = ModelTrainer()
mlflow_client = MLFlowClient(experiment_name="heart-disease-prediction")

INFO:src.model_training:ModelTrainer inicializado
INFO:src.mlflow_utils:✅ Experimento existente: heart-disease-prediction
INFO:src.mlflow_utils:MLFlow Client inicializado: http://mlflow:5000


# Fase 1: Modelos do Artigo

KNN, Random Forest, Logistic Regression, SVM, Gaussian NB, Decision Tree

## Treinar modelos do artigo

In [24]:
article_models = trainer.train_article_models(X_train, y_train)

print(f"\n✅ {len(article_models)} modelos do artigo treinados!")
for name in article_models.keys():
    print(f"   - {name}")

INFO:src.model_training:🔄 Treinando modelos do artigo...
INFO:src.model_training:🔄 Treinando 6 modelos...
INFO:src.model_training:🤖 Modelo criado: knn com params: {'n_neighbors': 5}
INFO:src.model_training:🔄 Treinando knn...
INFO:src.model_training:✅ knn treinado! Score treino: 0.8771
INFO:src.model_training:🤖 Modelo criado: random_forest com params: {'n_estimators': 100, 'random_state': 42, 'max_depth': 10}
INFO:src.model_training:🔄 Treinando random_forest...
INFO:src.model_training:✅ random_forest treinado! Score treino: 0.9900
INFO:src.model_training:🤖 Modelo criado: logistic_regression com params: {'max_iter': 1000, 'random_state': 42, 'C': 1.0}
INFO:src.model_training:🔄 Treinando logistic_regression...
INFO:src.model_training:✅ logistic_regression treinado! Score treino: 0.8538
INFO:src.model_training:🤖 Modelo criado: svm com params: {'kernel': 'rbf', 'probability': True, 'random_state': 42, 'C': 1.0, 'gamma': 'scale'}
INFO:src.model_training:🔄 Treinando svm...
INFO:src.model_trai


✅ 6 modelos do artigo treinados!
   - knn
   - random_forest
   - logistic_regression
   - svm
   - naive_bayes
   - decision_tree


# Fase 2: Modelos de Melhoria

Gradient Boosting, Random Forest tuned, XGBoost

## Treinar modelos de melhoria

In [25]:
improved_models = trainer.train_improved_models(X_train, y_train)

print(f"\n✅ {len(improved_models)} modelos de melhoria treinados!")
for name in improved_models.keys():
    print(f"   - {name}")

INFO:src.model_training:🔄 Treinando modelos de melhoria...
INFO:src.model_training:✅ XGBoost treinado!
INFO:src.model_training:🔄 Treinando 2 modelos...
INFO:src.model_training:🤖 Modelo criado: gradient_boosting com params: {'n_estimators': 100, 'random_state': 42, 'learning_rate': 0.1, 'max_depth': 5}
INFO:src.model_training:🔄 Treinando gradient_boosting...
INFO:src.model_training:✅ gradient_boosting treinado! Score treino: 1.0000
INFO:src.model_training:🤖 Modelo criado: random_forest_tuned com params: {'n_estimators': 200, 'random_state': 42, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1}
INFO:src.model_training:🔄 Treinando random_forest_tuned...
INFO:src.model_training:✅ random_forest_tuned treinado! Score treino: 1.0000
INFO:src.model_training:✅ 2 modelos treinados com sucesso!



✅ 2 modelos de melhoria treinados!
   - gradient_boosting
   - random_forest_tuned


# Combinar todos os modelos

In [26]:
all_models = {**article_models, **improved_models}

print(f"\n📊 Total de modelos treinados: {len(all_models)}")


📊 Total de modelos treinados: 8


# Validação Cruzada

## Cross-validation para cada modelo

In [27]:
cv_results = {}

for name, model in all_models.items():
    print(f"\n🔄 Cross-validation: {name}")
    cv_result = trainer.cross_validate(model, X_train, y_train, cv=5)
    cv_results[name] = cv_result

INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...
INFO:src.model_training:✅ CV Score: 0.8289 (+/- 0.0319)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...



🔄 Cross-validation: knn

🔄 Cross-validation: random_forest


INFO:src.model_training:✅ CV Score: 0.8439 (+/- 0.0340)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...
INFO:src.model_training:✅ CV Score: 0.8405 (+/- 0.0402)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...



🔄 Cross-validation: logistic_regression

🔄 Cross-validation: svm


INFO:src.model_training:✅ CV Score: 0.8455 (+/- 0.0336)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...
INFO:src.model_training:✅ CV Score: 0.6676 (+/- 0.0739)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...
INFO:src.model_training:✅ CV Score: 0.8173 (+/- 0.0118)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...



🔄 Cross-validation: naive_bayes

🔄 Cross-validation: decision_tree

🔄 Cross-validation: gradient_boosting


INFO:src.model_training:✅ CV Score: 0.8256 (+/- 0.0132)
INFO:src.model_training:🔄 Validação cruzada (5 folds, accuracy)...



🔄 Cross-validation: random_forest_tuned


INFO:src.model_training:✅ CV Score: 0.8456 (+/- 0.0332)


# Comparar resultados CV

In [28]:
cv_df = pd.DataFrame({
    'model': cv_results.keys(),
    'cv_mean': [r['mean'] for r in cv_results.values()],
    'cv_std': [r['std'] for r in cv_results.values()]
}).sort_values('cv_mean', ascending=False)

print("\n📊 Resultados Cross-Validation:")
display(cv_df)


📊 Resultados Cross-Validation:


,model,cv_mean,cv_std
7,random_forest_tuned,0.845565,0.033183
3,svm,0.845496,0.033600
1,random_forest,0.843898,0.033956
2,logistic_regression,0.840455,0.040154
0,knn,0.828898,0.031906
6,gradient_boosting,0.825551,0.013209
5,decision_tree,0.817273,0.011789
4,naive_bayes,0.667617,0.073856


# Salvar Modelos no MLFlow

In [29]:
# %% Registrar modelos no MLFlow (COM backup no MinIO)
for name, model in all_models.items():
    print(f"\n🔄 Salvando: {name}")
    
    # Iniciar run
    run = mlflow_client.start_run(run_name=f"train_{name}")
    
    # Log parâmetros
    if hasattr(model, 'get_params'):
        mlflow_client.log_params(model.get_params())
    
    # Log métricas de CV
    if name in cv_results:
        mlflow_client.log_metrics({
            'cv_mean_accuracy': cv_results[name]['mean'],
            'cv_std_accuracy': cv_results[name]['std']
        })
    
    # Log modelo NO MLFlow + Backup no MinIO
    run_id = mlflow_client.log_model(
        model, 
        artifact_path="model", 
        registered_model_name=name,
        save_to_s3=True,
        s3_client=s3
    )
    
    # Finalizar run
    mlflow_client.end_run()
    
    print(f"✅ {name} salvo (run_id: {run_id[:8]}...)")

print("\n✅ Todos os modelos salvos!")

# Verificar o que foi salvo no MinIO
print("\n📦 Modelos no MinIO:")
model_files = s3.list_files('models/')
for f in model_files:
    print(f"  - {f}")

INFO:src.mlflow_utils:🚀 Run iniciado: 5459f567315b464a9b3126f07aaaf2be
INFO:src.mlflow_utils:📝 Parâmetros registrados: 8 itens
INFO:src.mlflow_utils:📊 Métricas registradas: 2 itens
INFO:src.s3_utils:✅ Modelo salvo: s3://ml-bucket-heart/models/knn/5459f567315b464a9b3126f07aaaf2be.pkl
INFO:src.mlflow_utils:💾 Backup salvo no MinIO: models/knn/5459f567315b464a9b3126f07aaaf2be.pkl
2025/12/01 16:58:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'



🔄 Salvando: knn


INFO:src.mlflow_utils:✅ Run finalizado
INFO:src.mlflow_utils:🚀 Run iniciado: 492ef7d423d04f98b0d5714d181f089e


🏃 View run train_knn at: http://mlflow:5000/#/experiments/1/runs/5459f567315b464a9b3126f07aaaf2be
🧪 View experiment at: http://mlflow:5000/#/experiments/1
✅ knn salvo (run_id: 5459f567...)

🔄 Salvando: random_forest


INFO:src.mlflow_utils:📝 Parâmetros registrados: 19 itens
INFO:src.mlflow_utils:📊 Métricas registradas: 2 itens
INFO:src.s3_utils:✅ Modelo salvo: s3://ml-bucket-heart/models/random_forest/492ef7d423d04f98b0d5714d181f089e.pkl
INFO:src.mlflow_utils:💾 Backup salvo no MinIO: models/random_forest/492ef7d423d04f98b0d5714d181f089e.pkl
2025/12/01 16:58:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'
INFO:src.mlflow_utils:✅ Run finalizado
INFO:src.mlflow_utils:🚀 Run iniciado: 7a52894d78744d85933381d1dd6f537d
INFO:src.mlflow_utils:📝 Parâmetros registrados: 15 itens
INFO:src.mlflow_utils:📊 Métricas registradas: 2 itens
INFO:src.s3_utils:✅ Modelo salvo: s3://ml-bucket-heart/models/logistic_regression/7a52894d78744d85933381d1dd6f537d.pkl
INFO:src.mlflow_utils:💾 Backup 

...

🏃 View run train_naive_bayes at: http://mlflow:5000/#/experiments/1/runs/d3402fda1dc84e41a40cfe78ce86926a
🧪 View experiment at: http://mlflow:5000/#/experiments/1
✅ naive_bayes salvo (run_id: d3402fda...)

🔄 Salvando: decision_tree


INFO:src.mlflow_utils:🚀 Run iniciado: e0f42c9e61ad4b019bbabf65d0643ced
INFO:src.mlflow_utils:📝 Parâmetros registrados: 13 itens
INFO:src.mlflow_utils:📊 Métricas registradas: 2 itens
INFO:src.s3_utils:✅ Modelo salvo: s3://ml-bucket-heart/models/decision_tree/e0f42c9e61ad4b019bbabf65d0643ced.pkl
INFO:src.mlflow_utils:💾 Backup salvo no MinIO: models/decision_tree/e0f42c9e61ad4b019bbabf65d0643ced.pkl
2025/12/01 16:58:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'
INFO:src.mlflow_utils:✅ Run finalizado
INFO:src.mlflow_utils:🚀 Run iniciado: aee674d3cef54d488a364574701e5273
INFO:src.mlflow_utils:📝 Parâmetros registrados: 20 itens
INFO:src.mlflow_utils:📊 Métricas registradas: 2 itens
INFO:src.s3_utils:✅ Modelo salvo: s3://ml-bucket-heart/models/gradient_boosting

🏃 View run train_decision_tree at: http://mlflow:5000/#/experiments/1/runs/e0f42c9e61ad4b019bbabf65d0643ced
🧪 View experiment at: http://mlflow:5000/#/experiments/1
✅ decision_tree salvo (run_id: e0f42c9e...)

🔄 Salvando: gradient_boosting
🏃 View run train_gradient_boosting at: http://mlflow:5000/#/experiments/1/runs/aee674d3cef54d488a364574701e5273
🧪 View experiment at: http://mlflow:5000/#/experiments/1
✅ gradient_boosting salvo (run_id: aee674d3...)

🔄 Salvando: random_forest_tuned


<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'
INFO:src.mlflow_utils:✅ Run finalizado
INFO:src.s3_utils:📋 8 arquivos encontrados com prefixo 'models/'


🏃 View run train_random_forest_tuned at: http://mlflow:5000/#/experiments/1/runs/b432c9370dd243bca8de80d952617803
🧪 View experiment at: http://mlflow:5000/#/experiments/1
✅ random_forest_tuned salvo (run_id: b432c937...)

✅ Todos os modelos salvos!

📦 Modelos no MinIO:
  - models/decision_tree/e0f42c9e61ad4b019bbabf65d0643ced.pkl
  - models/gradient_boosting/aee674d3cef54d488a364574701e5273.pkl
  - models/knn/5459f567315b464a9b3126f07aaaf2be.pkl
  - models/logistic_regression/7a52894d78744d85933381d1dd6f537d.pkl
  - models/naive_bayes/d3402fda1dc84e41a40cfe78ce86926a.pkl
  - models/random_forest/492ef7d423d04f98b0d5714d181f089e.pkl
  - models/random_forest_tuned/b432c9370dd243bca8de80d952617803.pkl
  - models/svm/46375a6695cb4d8b95704b4ff6f5bc36.pkl


# Melhor Modelo

In [32]:
best_model_name = cv_df.iloc[0]['model']
best_cv_score = cv_df.iloc[0]['cv_mean']

print(f"\n🏆 Melhor modelo (CV): {best_model_name}")
print(f"   Accuracy: {best_cv_score:.4f}")

trainer.save_best_model(all_models[best_model_name], best_model_name)

INFO:src.model_training:🏆 Melhor modelo salvo: random_forest_tuned
INFO:src.model_training:🏆 Melhor modelo salvo: random_forest_tuned



🏆 Melhor modelo (CV): random_forest_tuned
   Accuracy: 0.8456
